# Test the final_model.pth on the Test set
Sets up the model, and loads the state dictionary defined by final_model.pth

### Imports

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.transforms.v2 as transforms
import torch.nn as nn

### Setup Device

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Setup transforms and data loader

In [6]:
# Define the transformations
transformations1 = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Resize((250, 250)),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

testing_dataset = torchvision.datasets.Flowers102(root='./data', split="test",
                                                  download=True, transform=transformations1)

test_loader = DataLoader(testing_dataset, batch_size=8, shuffle=False)

### Define the Model

In [7]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.flatten = nn.Flatten()
    self.relu = nn.PReLU()
    
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(16),
        nn.PReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(p=0.1)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(32),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(64),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer4 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(128),
        nn.PReLU(),
        nn.MaxPool2d(2, 2)
    )
    self.layer5 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(p=0.1)
    )
    self.layer6 = nn.Sequential(
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=(1,1), padding=(1,1)),
        nn.BatchNorm2d(256),
        nn.PReLU(),
        nn.MaxPool2d(2, 2),
        nn.Dropout(p=0.1)
    )
    
    self.fc1 = nn.Linear(256 * 3 * 3, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 102)  # Output layer for 102 classes

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.layer5(x)
    x = self.layer6(x)
    
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    return x


### Setup Network Accuracy on Testing function

In [8]:
def NetworkAccuracyOnTesting(model):
    model.eval()
    total_correct = 0
    total_samples = 0
    num_class_correct = [0] * 102
    num_class_samples = [0] * 102
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predictions = outputs.max(1)
            total_samples += labels.size(0)
            total_correct += predictions.eq(labels).sum().item()
            
            c = (predictions == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                num_class_correct[label] += c[i].item()
                num_class_samples[label] += 1

            # for i in range(len(labels)):
            #     label = labels[i]
            #     pred = predictions[i]
            #     if (label == pred):
            #         num_class_correct[label] += 1
            #     num_class_samples[label] += 1

    acc = 100.0 * total_correct / total_samples
    print(f'Accuracy on testing set: {acc} %')

    for i in range(102):
        acc = 100.0 * num_class_correct[i] / num_class_samples[i]
        print(f'Accuracy of {i} : {acc} %')

### Setup the model, load the state dictionary and run the testing function

In [9]:
model = CNN().to(device)
model.load_state_dict(torch.load('final_model.pth'))
NetworkAccuracyOnTesting(model)

Accuracy on testing set: 69.88128150918848 %
Accuracy of 0 : 80.0 %
Accuracy of 1 : 80.0 %
Accuracy of 2 : 20.0 %
Accuracy of 3 : 19.444444444444443 %
Accuracy of 4 : 84.44444444444444 %
Accuracy of 5 : 80.0 %
Accuracy of 6 : 70.0 %
Accuracy of 7 : 86.15384615384616 %
Accuracy of 8 : 69.23076923076923 %
Accuracy of 9 : 84.0 %
Accuracy of 10 : 29.850746268656717 %
Accuracy of 11 : 88.05970149253731 %
Accuracy of 12 : 86.20689655172414 %
Accuracy of 13 : 85.71428571428571 %
Accuracy of 14 : 93.10344827586206 %
Accuracy of 15 : 71.42857142857143 %
Accuracy of 16 : 87.6923076923077 %
Accuracy of 17 : 62.903225806451616 %
Accuracy of 18 : 44.827586206896555 %
Accuracy of 19 : 63.888888888888886 %
Accuracy of 20 : 85.0 %
Accuracy of 21 : 66.66666666666667 %
Accuracy of 22 : 76.05633802816901 %
Accuracy of 23 : 72.72727272727273 %
Accuracy of 24 : 90.47619047619048 %
Accuracy of 25 : 61.904761904761905 %
Accuracy of 26 : 75.0 %
Accuracy of 27 : 69.56521739130434 %
Accuracy of 28 : 81.03448275